In [1]:
import cv2
from PIL import Image
import numpy as np
import math
from processer import equalize, zmIceColor, extractGreen
data_path = "..\\dataset\\classifer\\train\\Black-grass\\adwstlhujk.png"

In [3]:
img = cv2.imread(data_path)
cv2.imshow('original', img)
# 颜色增强
ace_img = zmIceColor(img/255.0, ratio=4, radius=3)
cv2.imshow('ace43', ace_img)
ace_img  = ace_img * 255
ace_img = np.uint8(ace_img)
# ace_img = ace_img.astype(np.uint8)
gre_img = extractGreen(ace_img, 7)
cv2.imshow('gre_img', gre_img)
# 创建SIFT特征检测器
sift_after = cv2.SIFT_create()
# 特征点提取与描述子生成
kp, des = sift_after.detectAndCompute(gre_img, None)
# kp1, des1 = sift_after.detectAndCompute(gre_img, None)
# print(kp == kp1)
# print(kp, des)
show_img = gre_img.copy()
cv2.drawKeypoints(gre_img, kp, show_img)
cv2.imshow("des", show_img)
# cv2.drawKeypoints(gre_img, kp1, show_img)
# cv2.imshow("des1", show_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
np.set_printoptions(threshold=150)

In [11]:
des.shape

(67, 128)

In [32]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
datasets_path = "..\\dataset\\classifer\\train"

In [33]:
dataset = torchvision.datasets.ImageFolder(datasets_path, transform=transforms.ToTensor())

In [40]:
dataloader = DataLoader(dataset)

In [46]:
min_shape = 400
for x, y in dataloader:
    # print(x.shape, y)
    # break
    if x.shape[-1] <= min_shape:
        min_shape = x.shape[-1]
        print(x.shape)

torch.Size([1, 3, 334, 334])
torch.Size([1, 3, 307, 307])
torch.Size([1, 3, 117, 117])
torch.Size([1, 3, 88, 88])
torch.Size([1, 3, 78, 78])
torch.Size([1, 3, 74, 74])
torch.Size([1, 3, 74, 74])
torch.Size([1, 3, 73, 73])
torch.Size([1, 3, 58, 58])
torch.Size([1, 3, 54, 54])
torch.Size([1, 3, 51, 51])
torch.Size([1, 3, 51, 51])
torch.Size([1, 3, 49, 49])
torch.Size([1, 3, 49, 49])


In [49]:
shape_list = []
for x, y in dataloader:
    shape_list.append(x.shape[-1])

In [51]:
import statistics

median = statistics.median(shape_list)
print("中位数为：", median)
avg = statistics.mean(shape_list)
print("平均数为：", avg)
var = statistics.variance(shape_list)
print("方差为：", var)

中位数为： 264.0
平均数为： 355.815990990991
方差为： 91494.17406277082


In [47]:
avg_shape = 0
sum_shape = 0
num_count = 0
for x, y in dataloader:
    sum_shape += x.shape[-1]
    num_count += 1
avg_shape = sum_shape / num_count
avg_shape

355.815990990991

In [48]:
sum_shape

1579823

In [53]:
def processer(img):
    res_img = cv2.resize(img, (256, 256))
    # 颜色增强
    ace_img = zmIceColor(res_img/255.0, ratio=4, radius=3)
    ace_img  = ace_img * 255
    ace_img = np.uint8(ace_img)
    # 抽绿色
    gre_img = extractGreen(ace_img, 7)
    return gre_img

In [91]:
data_path1 = "..\\dataset\\classifer\\train\\Small-flowered Cranesbill\\zylncejrss.png"
img = cv2.imread(data_path1)
gre_img = processer(img)
cv2.imshow("gre_img", gre_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [92]:
# 创建SIFT特征检测器
sift_after = cv2.SIFT_create()
# 特征点提取与描述子生成
kp, des = sift_after.detectAndCompute(gre_img, None)
show_img = gre_img.copy()
cv2.drawKeypoints(gre_img, kp, show_img)
cv2.imshow("des", show_img)
# cv2.drawKeypoints(gre_img, kp1, show_img)
# cv2.imshow("des1", show_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [89]:
des.shape

(96, 128)

In [98]:
from skimage.feature import hog
gray_img = cv2.cvtColor(gre_img,cv2.COLOR_BGR2GRAY)
# b, g, r = cv2.split(gre_img)
# b_feature, b_img = hog(b, orientations=9, pixels_per_cell=(6, 6), cells_per_block=(3, 3), visualize=True, feature_vector=True)
# g_feature, g_img = hog(g, orientations=9, pixels_per_cell=(6, 6), cells_per_block=(3, 3), visualize=True, feature_vector=True)
# r_feature, r_img = hog(r, orientations=9, pixels_per_cell=(6, 6), cells_per_block=(3, 3), visualize=True, feature_vector=True)
# hog_feature, hog_img = hog(gray_img, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=True)
hog_feature, hog_img = hog(gray_img, orientations=9, pixels_per_cell=(6, 6), cells_per_block=(3, 3), visualize=True, feature_vector=False)
# hog_img = cv2.merge((b_img, g_img, r_img))
# cv2.imshow("gray_img", gray_img)
cv2.imshow("hog_img", hog_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
gray_img.shape

(256, 256)

In [97]:
hog_feature.shape

(40, 40, 3, 3, 9)

In [32]:
type(gre_img)

numpy.ndarray

In [36]:
gre_img.shape

(334, 334, 3)

In [90]:
from skimage.feature import local_binary_pattern
method = 'var'
n_points = 8
radius = 4
b, g, r = cv2.split(gre_img)
b = local_binary_pattern(b, n_points, radius, method)
g = local_binary_pattern(g, n_points, radius, method)
r = local_binary_pattern(r, n_points, radius, method)
feature_lbp = cv2.merge((b, g, r))
cv2.imshow("feature_lbp", feature_lbp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [87]:
feature_lbp.shape

(256, 256, 3)

In [7]:
des.shape

(67, 128)

In [14]:
bow_kmeans_trainer = cv2.BOWKMeansTrainer(20)
bow_kmeans_trainer.add(des)
# k-means返回词汇字典(聚类中心)
voc = bow_kmeans_trainer.cluster()
print(voc, type(voc),voc.shape)

[[42.333336   29.666668    0.         ... 14.666667    3.6666667
  25.333334  ]
 [ 0.          0.          0.         ...  0.33333334  1.
   6.3333335 ]
 [17.57143    11.142858    1.8571429  ...  4.857143    2.857143
   6.1428576 ]
 ...
 [12.5         0.          0.         ... 14.          0.
   0.5       ]
 [ 0.          0.          0.         ... 10.         23.5
  28.        ]
 [ 1.         29.         92.         ...  0.          0.
   0.        ]] <class 'numpy.ndarray'> (20, 128)


In [15]:
# FLANN匹配  
# algorithm用来指定匹配所使用的算法，可以选择的有LinearIndex、KTreeIndex、KMeansIndex、CompositeIndex和AutotuneInde
# 这里选择的是KTreeIndex(使用kd树实现最近邻搜索)
flann_params = dict(algorithm=1,tree=5)
flann = cv2.FlannBasedMatcher(flann_params,{})

print(flann)

< cv2.FlannBasedMatcher 0000012443197530>


In [16]:
#初始化bow提取器(设置词汇字典),用于提取每一张图像的BOW特征描述
sift = cv2.SIFT_create()
bow_img_descriptor_extractor = cv2.BOWImgDescriptorExtractor(sift, flann)        
bow_img_descriptor_extractor.setVocabulary(voc)

print(bow_img_descriptor_extractor)

< cv2.BOWImgDescriptorExtractor 00000124431941F0>


In [17]:
feature_bow_list = [] 
sift = cv2.SIFT_create()
image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
feature_bow = bow_img_descriptor_extractor.compute(image, sift.detect(image))
feature_bow

array([[0.01754386, 0.06432749, 0.0497076 , 0.07602339, 0.0497076 ,
        0.00877193, 0.03508772, 0.03508772, 0.00877193, 0.05847953,
        0.06140351, 0.02923977, 0.01461988, 0.00877193, 0.05555556,
        0.03508772, 0.19590643, 0.0497076 , 0.12280702, 0.02339181]],
      dtype=float32)

In [ ]:
import numpy as np
import cv2

class BOW(object):
    
    def __init__(self,):
        #创建一个SIFT对象  用于关键点提取
        self.feature_detector  = cv2.xfeatures2d.SIFT_create()
        #创建一个SIFT对象  用于关键点描述符提取 
        self.descriptor_extractor = cv2.xfeatures2d.SIFT_create()

    def path(self,cls,i):
        \'\'\'
        用于获取图片的全路径 
        \'\'\'
        return \'%s/%s/%s.%d.jpg\'%(self.train_path,cls,cls,i+1)

    def fit(self,train_path,k):
        \'\'\'
        开始训练
        
        args：
            train_path：训练集图片路径  我们使用的数据格式为 train_path/dog/dog.i.jpg  train_path/cat/cat.i.jpg
            k：k-means参数k
        \'\'\'
        self.train_path = train_path        
        
        #FLANN匹配  参数algorithm用来指定匹配所使用的算法，可以选择的有LinearIndex、KTreeIndex、KMeansIndex、CompositeIndex和AutotuneIndex，这里选择的是KTreeIndex(使用kd树实现最近邻搜索)
        flann_params = dict(algorithm=1,tree=5)
        flann = cv2.FlannBasedMatcher(flann_params,{})
        
        #创建BOW训练器，指定k-means参数k   把处理好的特征数据全部合并，利用聚类把特征词分为若干类，此若干类的数目由自己设定，每一类相当于一个视觉词汇
        bow_kmeans_trainer = cv2.BOWKMeansTrainer(k)
        
        pos = \'dog\'
        neg = \'cat\'
        
        #指定用于提取词汇字典的样本数
        length = 10
        #合并特征数据  每个类从数据集中读取length张图片(length个狗,length个猫)，通过聚类创建视觉词汇
        for i in range(length):
            bow_kmeans_trainer.add(self.sift_descriptor_extractor(self.path(pos,i)))
            bow_kmeans_trainer.add(self.sift_descriptor_extractor(self.path(neg,i)))

        #进行k-means聚类，返回词汇字典 也就是聚类中心
        voc = bow_kmeans_trainer.cluster()
        
        #输出词汇字典  <class \'numpy.ndarray\'> (40, 128)
        print(type(voc),voc.shape)
        
        #初始化bow提取器(设置词汇字典),用于提取每一张图像的BOW特征描述
        self.bow_img_descriptor_extractor = cv2.BOWImgDescriptorExtractor(self.descriptor_extractor,flann)        
        self.bow_img_descriptor_extractor.setVocabulary(voc)
        
        
        #创建两个数组，分别对应训练数据和标签，并用BOWImgDescriptorExtractor产生的描述符填充
        #按照下面的方法生成相应的正负样本图片的标签 1：正匹配  -1：负匹配
        traindata,trainlabels = [],[]
        for i in range(400):   #这里取200张图像做训练
            traindata.extend(self.bow_descriptor_extractor(self.path(pos,i)))
            trainlabels.append(1)
            traindata.extend(self.bow_descriptor_extractor(self.path(neg,i)))
            trainlabels.append(-1)
         
        #创建一个SVM对象    
        self.svm = cv2.ml.SVM_create()
        #使用训练数据和标签进行训练
        self.svm.train(np.array(traindata),cv2.ml.ROW_SAMPLE,np.array(trainlabels))


    def predict(self,img_path):
        \'\'\'
        进行预测样本   
        \'\'\'
        #提取图片的BOW特征描述
        data = self.bow_descriptor_extractor(img_path)
        res = self.svm.predict(data)
        print(img_path,\'\t\',res[1][0][0])
        
        #如果是狗 返回True
        if res[1][0][0] == 1.0:
            return True
        #如果是猫，返回False
        else:
            return False
        
    
     
    def sift_descriptor_extractor(self,img_path):
        \'\'\'
        特征提取：提取数据集中每幅图像的特征点，然后提取特征描述符，形成特征数据(如：SIFT或者SURF方法)；
        \'\'\'
        im = cv2.imread(img_path,0)     
        return self.descriptor_extractor.compute(im,self.feature_detector.detect(im))[1]  
    
    
    def bow_descriptor_extractor(self,img_path):
        \'\'\'
        提取图像的BOW特征描述(即利用视觉词袋量化图像特征)
        \'\'\'
        im = cv2.imread(img_path,0)
        return self.bow_img_descriptor_extractor.compute(im,self.feature_detector.detect(im))
    


if __name__ == \'__main__\':
    #测试样本数量，测试结果
    test_samples = 100
    test_results = np.zeros(test_samples,dtype=np.bool)
    
    #训练集图片路径  狗和猫两类  进行训练
    train_path = \'./data/cat_and_dog/data/train\'
    bow = BOW()
    bow.fit(train_path,40)
    
    
    #指定测试图像路径
    for index in range(test_samples):
        dog = \'./data/cat_and_dog/data/train/dog/dog.{0}.jpg\'.format(index)
        dog_img = cv2.imread(dog)    
    
        #预测
        dog_predict = bow.predict(dog)    
        test_results[index] = dog_predict
        
    #计算准确率
    accuracy = np.mean(test_results.astype(dtype=np.float32)) 
    print(\'测试准确率为：\',accuracy)
        
    #可视化最后一个    
    font = cv2.FONT_HERSHEY_SIMPLEX    
    if test_results[0]:
        cv2.putText(dog_img,\'Dog Detected\',(10,30),font,1,(0,255,0),2,cv2.LINE_AA)
        
    cv2.imshow(\'dog_img\',dog_img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()